# Simulated Data Anonymisation Task from the Commonwealth Bank Virtual Experience on Forage

## Load the dataset

In [42]:
import pandas as pd  # Import pandas for data manipulation and analysis
import numpy as np   # Import NumPy for numerical operations and mathematical functions

In [43]:
# Load the dataset
df = pd.read_excel('mobile_customers.xlsx')
df.head(3)

,Unnamed: 0,customer_id,date_registered,username,name,gender,address,email,birthdate,current_location,residence,employer,job,age,salary,credit_card_provider,credit_card_number,credit_card_security_code,credit_card_expire
0,0,24c9d2d0-d0d3-4a90-9a3a-e00e4aac99bd,2021-09-29,robertsbryan,Jonathan Snyder,M,"24675 Susan Valley\nNorth Dianabury, MO 02475",marcus58@hotmail.com,1978-03-11,"['78.937112', '71.260464']","195 Brandi Junctions\nNew Julieberg, NE 63410","Byrd, Welch and Holt",Chief Technology Officer,49,53979,VISA 19 digit,38985874269846,994,2023-10-27 00:00:00
1,1,7b2bc220-0296-4914-ba46-d6cc6a55a62a,2019-08-17,egarcia,Susan Dominguez,F,"4212 Cheryl Inlet\nPort Davidmouth, NC 54884",alexanderkathy@hotmail.com,1970-11-29,"['-24.1692185', '100.746122']","58272 Brown Isle Apt. 698\nPort Michael, HI 04693",Hurst PLC,Data scientist,43,81510,Discover,6525743622515979,163,2023-07-30 00:00:00
2,2,06febdf9-07fb-4a1b-87d7-a5f97d9a5faf,2019-11-01,turnermegan,Corey Hebert,M,"07388 Coleman Prairie\nLake Amy, IA 78695",vwood@gmail.com,2009-04-23,"['8.019908', '-19.603269']","36848 Jones Lane Suite 282\nMarquezbury, ID 26822","Mora, Caldwell and Guerrero",Chief Operating Officer,47,205345,VISA 16 digit,4010729915028682247,634,2023-04-26 00:00:00


## Step 1 : Remove unwanted columns

In [44]:
# Remove columns that are irrelevant to the anonymisation process
df.drop(columns=['Unnamed: 0', 'customer_id', 'current_location'], inplace=True)
df.head(3)

,date_registered,username,name,gender,address,email,birthdate,residence,employer,job,age,salary,credit_card_provider,credit_card_number,credit_card_security_code,credit_card_expire
0,2021-09-29,robertsbryan,Jonathan Snyder,M,"24675 Susan Valley\nNorth Dianabury, MO 02475",marcus58@hotmail.com,1978-03-11,"195 Brandi Junctions\nNew Julieberg, NE 63410","Byrd, Welch and Holt",Chief Technology Officer,49,53979,VISA 19 digit,38985874269846,994,2023-10-27 00:00:00
1,2019-08-17,egarcia,Susan Dominguez,F,"4212 Cheryl Inlet\nPort Davidmouth, NC 54884",alexanderkathy@hotmail.com,1970-11-29,"58272 Brown Isle Apt. 698\nPort Michael, HI 04693",Hurst PLC,Data scientist,43,81510,Discover,6525743622515979,163,2023-07-30 00:00:00
2,2019-11-01,turnermegan,Corey Hebert,M,"07388 Coleman Prairie\nLake Amy, IA 78695",vwood@gmail.com,2009-04-23,"36848 Jones Lane Suite 282\nMarquezbury, ID 26822","Mora, Caldwell and Guerrero",Chief Operating Officer,47,205345,VISA 16 digit,4010729915028682247,634,2023-04-26 00:00:00


## Step 2: Masking 'username' column

In [45]:
# Data Masking for 'username' column
def mask_name(name):
    if len(name) <= 2:
        return '*' * len(name)
    return name[0] + '*' * (len(name) - 2) + name[-1]

df['username_mask'] = df['username'].apply(mask_name)

# Preview the result
print(df[['username', 'username_mask']].head(3))

       username username_mask
0  robertsbryan  r**********n
1       egarcia       e*****a
2   turnermegan   t*********n


## Step 3: Replace with fake 'name' column

In [46]:
# Replace original 'name' column with fake 'name' column
from faker import Faker

# Initialize Faker
fake = Faker()

# Set a seed for reproducibility
Faker.seed(42)

# Create a new column with fake names
df['name_fake'] = [fake.name() for _ in range(len(df))]

# Preview the result
print(df[['name', 'name_fake']].head(3))

              name        name_fake
0  Jonathan Snyder     Allison Hill
1  Susan Dominguez      Noah Rhodes
2     Corey Hebert  Angie Henderson


## Step 4: Masking 'email' column 

In [47]:
# Data Masking for 'email' column
def mask_email(email):
    name, domain = email.split('@')
    masked = name[0] + '*' * (len(name) - 2) + name[-1]
    return masked + '@' + domain

df['masked_email'] = df['email'].apply(mask_name)

# Preview the result
print(df[['email', 'masked_email']].head(3))

                        email                masked_email
0        marcus58@hotmail.com        m******************m
1  alexanderkathy@hotmail.com  a************************m
2             vwood@gmail.com             v*************m


## Step 5: Adding noise to 'date registered' & 'birthdate' columns

In [48]:
from datetime import timedelta
import random

# Convert date columns to datetime format
df['date_registered'] = pd.to_datetime(df['date_registered'], dayfirst=True, errors='coerce')
df['birthdate'] = pd.to_datetime(df['birthdate'], dayfirst=True, errors='coerce')

# Function to add random noise to a date (± up to 180 days)
def add_noise_to_date(date, max_days=180):
    if pd.isna(date):
        return np.nan
    noise = timedelta(days=random.randint(-max_days, max_days))
    return date + noise

# Apply the noise function to both columns
df['date_registered_noise'] = df['date_registered'].apply(lambda x: add_noise_to_date(x))
df['birthdate_noise'] = df['birthdate'].apply(lambda x: add_noise_to_date(x, max_days=120))  # less noise for birthdate

# Format dates to dd/mm/yyyy (or remove leading zeros like your example)
df['date_registered_noise'] = df['date_registered_noise'] .dt.strftime('%-d/%-m/%Y')
df['birthdate_noise'] = df['birthdate_noise'].dt.strftime('%-d/%-m/%Y')

# Preview the updated data
print(df[['date_registered', 'date_registered_noise','birthdate', 'birthdate_noise']].head(3))

  date_registered date_registered_noise  birthdate      birthdate_noise
0      2021-09-29   2021-10-06 00:00:00 1978-03-11  1978-03-06 00:00:00
1      2019-08-17   2019-03-13 00:00:00 1970-11-29  1971-01-01 00:00:00
2      2019-11-01   2020-03-24 00:00:00 2009-04-23  2009-05-13 00:00:00


## Step 6: Grouping 'age' & 'salary' columns

In [49]:
# Discretization/Bucketing for age and salary columns (grouping them)
# Define bins
bins1 = [15, 20, 30, 40, 50, 60, 65]

# Apply binning using pd.cut
df['age_bin'] = pd.cut(df['age'], bins1)

# Define bins
bins2 = [20000, 50000, 80000, 110000, 140000, 170000, 200000, 230000, 250000]

# Apply binning using pd.cut
df['salary_bin'] = pd.cut(df['salary'], bins2)

# Preview the updated data
print(df[['age', 'age_bin','salary', 'salary_bin']].head(3))

   age   age_bin  salary        salary_bin
0   49  (40, 50]   53979    (50000, 80000]
1   43  (40, 50]   81510   (80000, 110000]
2   47  (40, 50]  205345  (200000, 230000]


## Step 7: Hashing 'credit card provider' & 'credit card expire' columns

In [50]:
# Tokenising/Hashing for 'credit_card_provider' and 'credit_card_expire' columns
import hashlib

# Define hashing function (to keep token length short)
def hash_token(value):
    return hashlib.sha256(value.encode()).hexdigest()[:10]  # first 10 characters

# Apply hashing to each column
df['credit_card_provider_hash'] = df['credit_card_provider'].apply(lambda x: hash_token(str(x)))
df['credit_card_expire_hash'] = df['credit_card_expire'].apply(lambda x: hash_token(str(x)))

# Preview the updated data
print(df[['credit_card_provider_hash', 'credit_card_provider','credit_card_expire_hash', 'credit_card_expire']].head(3))

  credit_card_provider_hash credit_card_provider credit_card_expire_hash  \
0                ae568399e6        VISA 19 digit              ab867e0e1a   
1                d4a33d5b78             Discover              9366ba266b   
2                7e56c08964        VISA 16 digit              fb98978031   

    credit_card_expire  
0  2023-10-27 00:00:00  
1  2023-07-30 00:00:00  
2  2023-04-26 00:00:00  


## Step 8: Masking 'credit card number' & 'credit card security code' columns

In [51]:
# Randomize credit card numbers into float-like/scientific notation
def mask_credit_card_number(x):
    return np.random.uniform(-1e18, 5e18)

# Randomize CVV into float-like values
def mask_credit_card_cvv(x):
    return np.random.uniform(-1000, 8000)

# Apply to each column
df['credit_card_number_mask'] = df['credit_card_number'].apply(mask_credit_card_number)
df['credit_card_security_code_mask'] = df['credit_card_security_code'].apply(mask_credit_card_cvv)

# Preview the updated data
print(df[['credit_card_number_mask', 'credit_card_number','credit_card_security_code_mask', 'credit_card_security_code']].head(3))

   credit_card_number_mask   credit_card_number  \
0             9.433379e+17       38985874269846   
1             1.512243e+18     6525743622515979   
2             4.248557e+17  4010729915028682247   

   credit_card_security_code_mask  credit_card_security_code  
0                      -25.532522                        994  
1                     3428.818684                        163  
2                     4077.767953                        634  


## Step 9: Hashing 'employer' & 'job' columns

In [52]:
#Tokenization/Hashing for employer and job columns
import hashlib

# Hash function
def hash_text(text):
    return hashlib.sha256(text.encode()).hexdigest()[:10]  # Shorten if needed

# Apply hashing
df['employer_masked'] = df['employer'].apply(hash_text)
df['job_masked'] = df['job'].apply(hash_text)

# Preview the updated data
print(df[['employer_masked', 'employer','job_masked', 'job']].head(3))

  employer_masked                     employer  job_masked  \
0      a2e5631b9e         Byrd, Welch and Holt  847953c7c1   
1      a8a4a9f466                    Hurst PLC  5124dd1227   
2      ce5b9ed5f6  Mora, Caldwell and Guerrero  8d4277a3e9   

                        job  
0  Chief Technology Officer  
1            Data scientist  
2   Chief Operating Officer  


## Step 10: Replacing with fake 'residence' & 'address' columns

In [53]:
from faker import Faker

# Initialize Faker
fake = Faker()

# Optional: Set a seed for reproducibility
Faker.seed(42)

# Replace 'residence' column with fake addresses
df['residence_fake'] = [fake.address().replace("\n", " ") for _ in range(len(df))]

# Replace 'address' column with fake addresses
df['address_fake'] = [fake.address().replace("\n", " ") for _ in range(len(df))]

# Preview the updated data
print(df[['residence_fake', 'residence', 'address_fake', 'address']].head(3))

                                      residence_fake  \
0          433 Jill Springs New Roberttown, CO 29158   
1      386 Shane Harbors Port Lindachester, MA 36922   
2  16155 Roman Stream Suite 816 New Kellystad, OK...   

                                           residence  \
0      195 Brandi Junctions\nNew Julieberg, NE 63410   
1  58272 Brown Isle Apt. 698\nPort Michael, HI 04693   
2  36848 Jones Lane Suite 282\nMarquezbury, ID 26822   

                                    address_fake  \
0  725 Mark Land Suite 139 West Taylor, SC 02491   
1        3580 Mark Summit Lawrencefurt, AS 83145   
2    19120 Daniel Run Apt. 944 Leeside, WI 99635   

                                         address  
0  24675 Susan Valley\nNorth Dianabury, MO 02475  
1   4212 Cheryl Inlet\nPort Davidmouth, NC 54884  
2      07388 Coleman Prairie\nLake Amy, IA 78695  


## Dropping/Removing original columns that not undergone data anonymisation

In [55]:
# Remove original columns that are before data anonymisation
df.drop(columns=['username', 'name', 'email', 'date_registered', 'birthdate', 'age', 'salary', 'credit_card_provider',
                 'credit_card_expire', 'credit_card_number', 'credit_card_security_code', 'employer', 'job', 'residence', 'address'], inplace=True)
df.head(3)

,gender,username_mask,name_fake,masked_email,date_registered_noise,birthdate_noise,age_bin,salary_bin,credit_card_provider_hash,credit_card_expire_hash,credit_card_number_mask,credit_card_security_code_mask,employer_masked,job_masked,residence_fake,address_fake
0,M,r**********n,Allison Hill,m******************m,2021-10-06 00:00:00,1978-03-06 00:00:00,"(40, 50]","(50000, 80000]",ae568399e6,ab867e0e1a,9.433379e+17,-25.532522,a2e5631b9e,847953c7c1,"433 Jill Springs New Roberttown, CO 29158","725 Mark Land Suite 139 West Taylor, SC 02491"
1,F,e*****a,Noah Rhodes,a************************m,2019-03-13 00:00:00,1971-01-01 00:00:00,"(40, 50]","(80000, 110000]",d4a33d5b78,9366ba266b,1.512243e+18,3428.818684,a8a4a9f466,5124dd1227,"386 Shane Harbors Port Lindachester, MA 36922","3580 Mark Summit Lawrencefurt, AS 83145"
2,M,t*********n,Angie Henderson,v*************m,2020-03-24 00:00:00,2009-05-13 00:00:00,"(40, 50]","(200000, 230000]",7e56c08964,fb98978031,4.248557e+17,4077.767953,ce5b9ed5f6,8d4277a3e9,"16155 Roman Stream Suite 816 New Kellystad, OK...","19120 Daniel Run Apt. 944 Leeside, WI 99635"


## Save cleaned dataset into new Excel

In [56]:
# Assume df is your cleaned DataFrame
df.to_excel("mobile_customers_cleaned.xlsx", index=False)